# Cyrite memory library

The cyrite library provides a few memory implementations that synthesize for most targets via HDL transfer. However synthesis tool specific issues come into play with generic memory models.

Therefore, only functionally correct HDL simulation and synthesis via yosys is currently maintained.
For pure engineering purposes, it is recommended to wrap external descriptions via a `@blackbox` entity.

## Simple dual port RAM

This RAM implementation can read/write from one port and read only from the other. It uses memory port containers as interface. There is also a dual clock version `sdpram_dc` present, but not documented here.

Import the sdpram module from the RAM library:

In [1]:
from cyhdl import *
from cyrite.library.ram import sdpram

We augment the memory port classes by a few simulation macros:

In [2]:
class SimMemPort_W(sdpram.MemPort_W):
	def __init__(self, ctx, ADDR_W = 12, DATA_W = 32):
		super().__init__(ctx, ADDR_W = ADDR_W, DATA_W = DATA_W)

	@cyrite_method.sequence
	def write(self, clk, addr, data):
		print("WRITE", addr, data)
		yield clk.negedge
		self.addr.next = addr
		self.idata.next = data
		self.we.next = True
		yield clk.negedge
		self.we.next = False

class SimMemPort_R(sdpram.MemPort_R):
	def __init__(self, ctx, ADDR_W = 12, DATA_W = 32):
		super().__init__(ctx, ADDR_W = ADDR_W, DATA_W = DATA_W)

	@cyrite_method.sequence
	def check_sequence(self, clk, sequence):
		for i in sequence:
			yield clk.negedge
			self.iaddr.next = i[0]
			yield clk.negedge
			assert q == i[1]



## Testbench

We use the test class below to carry out some tests on a `sdpram` unit. This one uses a single clock for both ports.

In [3]:
from myirl.library.memory import MemArray

class SDPTest(cyrite_factory.Module):
	"""Simple dual port RAM test suite. This uses:
	* a single clock for two ports
	* Asymmetrical dual read, single write"""

	def __init__(self, target, BRAM_W = 10, DATA_W = 8):
		super().__init__("sdptest", target)
		self.period = 10
		self.BRAM_W = BRAM_W
		self.DATA_W = DATA_W

	def create_data(self):
		ram_data = [ intbv(i)[self.DATA_W:] for i in range(2**self.BRAM_W) ]

		init_ram = MemArray(ram_data,
			name = 'ram_sig',
			init = True,
		elem_size = self.DATA_W)

		return init_ram
	
	@cyrite_factory.testbench("ns")
	def testbench(self, uut, TRANSPARENT):
		clk = self.ClkSignal(name = 'clk')

		pr = SimMemPort_R(self, ADDR_W = self.BRAM_W, DATA_W = self.DATA_W, name = "pr")
		pw = SimMemPort_W(self, ADDR_W = self.BRAM_W, DATA_W = self.DATA_W, name = "pw")

		@self.always(delay(self.period))
		def clkgen():
			clk.next = ~clk

		inst = uut(clk, port_r = pr, port_rw = pw, ram = self.create_data(),
			TRANSPARENT = TRANSPARENT)

		q = pr.odata

		@self.hdlmacro
		def transparency_check(addr, value):
			pr.addr.next = addr
			# Write and verify that we're not bypassing:
			# i.e. the value is expected to be the initial one
			yield from pw.write(clk, addr, value)
			if TRANSPARENT:
				assert q == value
			else:
				assert q == addr # Initialized value
			# Now make sure we're getting the written value
			yield clk.negedge
			assert q == value

		@self.sequence
		def main():
			pw.we.next = False
			yield delay(5)
			yield from transparency_check(0x20, 0xaa)
			yield delay(40)

			raise StopSimulation

		return instances()



Now we run the test bench through the icarus Verilog simulator:

In [4]:
from cyrite.simulation import icarus

def test_sdpram(target, TRANSPARENT):
    t = SDPTest(target)

    tb = t.testbench(sdpram.sdpram, TRANSPARENT)

    tb.run(200, debug = True, recompile = True, wavetrace = True)
    return tb

tb = test_sdpram(icarus.ICARUS, True)

 Declare obj 'testbench' in context '(SDPTest 'sdptest')'(<class '__main__.SDPTest'>) 
 Writing 'sdpram' to file /tmp/sdpram.v 
DEBUG: Source 'ram_sig' is logic: <class 'myirl.library.memory.MemArray'>
DEBUG NAME port_r_pr_out_odata <class 'myirl.kernel._types.ChildAlias'>
 Writing 'testbench' to file /tmp/testbench.v 
DEBUG NAME pw_in_we <class 'myirl.kernel._types.ChildAlias'>
DEBUG NAME pr_in_addr <class 'myirl.kernel._types.ChildAlias'>
DEBUG NAME pw_in_addr <class 'myirl.kernel._types.ChildAlias'>
DEBUG NAME pw_in_idata <class 'myirl.kernel._types.ChildAlias'>
 Note: Changing library path prefix to /tmp/ 
 Creating library file /tmp/module_defs.v 
DEBUG FILES ['/tmp/sdpram.v', '/tmp/testbench.v']
==== COSIM stdout ====
VCD info: dumpfile testbench.vcd opened for output.
WRITE          32         170 
Stop Simulation



/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/cyrite/simulation/icarus.py:51: UserWarning: Ignoring wavetrace argument for Verilog simulator
  warnings.warn("Ignoring wavetrace argument for Verilog simulator")


### Wave display

We can observe that transparency is sane:

In [5]:
from cyrite import waveutils
c = {
    "testbench.clk" : None, "testbench.pw_in_addr[9:0]"  : None,
    "testbench.pr_in_addr[9:0]"  : None,
    "testbench.pw_in_idata[7:0]" : None, "testbench.pw_in_we" : None,
    "testbench.pr_out_odata[7:0]" : None

}

waveutils.draw_wavetrace(tb, "testbench.vcd", "clk", cfg = c)

## True dual port RAM

See [True dual port RAM examples](../examples/tdpram.ipynb)

## Direct synthesis support for RAM

The RTLIL target conversion detects some behavioural descriptions of RAM and turns them into yosys primitives that can be processed further.
Due to the complex memory handling architecture of yosys, the entire RAM API is frozen to a possibly outdated version and supports CXXRTL output in particular. All other FPGA architecture specific configurations were removed due to maintenance complexity.

As an example to verify proper conversion to your target, see:

* [Yosys memory inference](../examples/yosys_memory.ipynb)

The recommended approach to implement a design for a specific RAM architecture is to use `@blackbox` wrappers for an existing working Verilog model or a primitive.